In [1]:
#!pip install --quiet langchain-anthropic langchain-neo4j cyVer langchain-google-genai json-repair "numpy<2"

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from tqdm import tqdm
import pandas as pd

from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

from utils import (
    _value_sanitize,
    extract_json_from_markdown,
    sampling_query,
    validate_cypher,
    process_database,
    process_all_examples_with_limit,
    convert_datetime
)
from prompts import (
    system_prompt,
    simple_system_prompt,
)

# Generate dataset

In [4]:
# LLM selection
models = [ChatAnthropic(model='claude-opus-4-20250514'), ChatGoogleGenerativeAI(model="gemini-2.5-pro")]

# Database selection (for demo database)
db_url = "neo4j+s://demo.neo4jlabs.com"
databases = [
    "companies",
    "twitch", 
    "network",
    "northwind",
    "ClinicalKnowledgeGraph"
]


In [5]:
simple_batch_count = 1 # Number of iterations for simple queries
multi_batch_count = 3 # Number of iterations complex queries

output = []

for model in models:
    print(model.model)
    for credential in tqdm(databases, desc="Processing databases"):
        # Simple question
        database_records = process_database(
            credential, db_url, model, simple_batch_count, simple_system_prompt
        )
        output.extend(database_records)

        database_records = process_database(
            credential, db_url, model, multi_batch_count, system_prompt
        )
        output.extend(database_records)

claude-opus-4-20250514


Iterations for companies: 100%|████████████████████| 1/1 [02:36<00:00, 156.98s/it]
                                                                                  
Iterations for twitch: 100%|███████████████████████| 1/1 [02:26<00:00, 146.25s/it]
                                                                                  
Iterations for network: 100%|██████████████████████| 1/1 [02:40<00:00, 160.22s/it]
                                                                                  
Iterations for northwind: 100%|████████████████████| 1/1 [02:15<00:00, 135.06s/it]
                                                                                  
Iterations for ClinicalKnowledgeGraph: 100%|███████| 1/1 [05:50<00:00, 350.89s/it]
                                                                                  
Processing databases: 100%|█████████████████████| 5/5 [2:02:17<00:00, 1467.40s/it]


models/gemini-2.5-pro


Iterations for companies: 100%|████████████████████| 1/1 [02:36<00:00, 156.71s/it]
                                                                                  
Iterations for companies:  33%|██████▋             | 1/3 [06:13<12:26, 373.36s/it]Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: category)} {position: line: 1, column: 180, offset: 179} for query: "MATCH (ibm:Organization {name: 'IBM'})<-[:HAS_SUPPLIER]-(sup:Organization)-[:HAS_INVESTOR]->(inv:Organization) WHERE (sup)-[:HAS_CATEGORY]->(:IndustryCategory) AND any(cat IN sup.category WHERE cat.name CONTAINS 'Technology') RETURN inv.name 

# Generate text answers

In [6]:
qa_model = ChatAnthropic(model='claude-3-5-haiku-latest')

In [7]:
# Generate text-based answers
await process_all_examples_with_limit(output, qa_model)

Processing examples: 100%|████████████████████| 1982/1982 [04:12<00:00,  7.86it/s]


In [8]:
df = pd.DataFrame.from_records(output)
df.head(5)

,question,cypher,query_type,complexity,noise_applied,model,validated,result,answer,noise_type
0,What's Microsoft Corporation's revenue?,MATCH (o:Organization {name: 'Microsoft Corpor...,Direct Property Access,0-hop,False,anthropic-chat,True,[{'company_revenue': 198270000000.0}],Microsoft Corporation's revenue is $198.27 bil...,NaN
1,How many employees does SAP have?,MATCH (o:Organization {name: 'SAP'}) RETURN o....,Direct Property Access,0-hop,False,anthropic-chat,True,[{'employee_count': 111961}],"SAP has 111,961 employees.",NaN
2,What's Mircosoft's motto?,MATCH (o:Organization {name: 'Microsoft Corpor...,Direct Property Access,0-hop,True,anthropic-chat,True,[{'company_motto': 'We’re on a mission to empo...,"Microsoft's motto is: ""We're on a mission to e...",typo
3,Is Cloud Sherpas still active?,MATCH (o:Organization {name: 'Cloud Sherpas'})...,Direct Property Access,0-hop,False,anthropic-chat,True,[{'is_dissolved': True}],"Based on the query result, Cloud Sherpas is no...",NaN
4,Tell me IBM's summary,MATCH (o:Organization {name: 'IBM'}) RETURN o....,Direct Property Access,0-hop,False,anthropic-chat,True,[{'company_summary': 'American multinational t...,IBM is an American multinational technology an...,NaN


In [14]:
import pandas as pd
import json
from datetime import datetime
import neo4j

def convert_datetime(obj):
    if isinstance(obj, (pd.Timestamp, datetime, neo4j.time.DateTime)):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

# Assuming 'output' is defined elsewhere in your code
with open("generated_dataset.json", "w") as f:
    json.dump(output, f, indent=2, default=convert_datetime)